# Inteligentnta wyszukiwarka

W pierwszym przypadku użycia wykorzystamy wektorową bazę danych do stworzenia inteligentnej wyszukiwarki tekstowej. Nie będzie ona działać na wyszukiwaniu podobnych ciągów znaków,
a zamiast tego będzie opierać się na wektorowej reprezentacji tekstu. Mocno upraszczając - z każdego tekstu da się stworzyć wektor, który reprezentuje użyte
w tekście słowa. Wektor ten, umieszczony w wielowymiarowej przestrzeni, reprezentuje znaczenie tekstu.

Poniższe komendy instalują bilbiotekę *Sentence Transformers* ([sbert.net](https://www.sbert.net/)), która posiada wbudowane modele potrafiące przekształcać tekst na wektor.
Moglibyśmy po prostu wykonać `pip install sentence-transformers`, ale ta komenda pobrałaby wszystkie zależności, w tym możliwość uruchamiania transformerów na GPU, czego nie potrzebujemy
w trakcie laboratorium. Dzięki pominięciu tych zależności oszczędzimy pobieranie około 2 GB.

In [1]:
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
%pip install transformers tqdm numpy scikit-learn scipy nltk sentencepiece
%pip install --no-deps sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.6/174.6 MB 40.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 65.7 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 44.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0

Od teraz możemy używać transformera, by przekształcić dowolny tekst w wektor. Użyjemy do tego wytrenowanego modelu 
[all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

In [2]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("all-MiniLM-L6-v2")

encoder.encode("Ala zostawiła kota w domu, bo uczy się korzystać z wektorowych baz danych.")

/usr/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


array([-5.25917932e-02,  1.18866071e-01, -2.19975654e-02,  1.13960300e-02,
       -1.04046628e-01, -2.54100226e-02,  7.86777958e-03,  2.90198922e-02,
       -3.60894129e-02, -2.05658041e-02, -1.12653812e-02,  2.12511420e-02,
       -1.00599453e-02,  3.80982272e-02, -2.99162026e-02, -2.13984530e-02,
       -4.74220403e-02,  8.73931199e-02, -5.98045550e-02, -1.41044836e-02,
       -3.67920520e-03, -2.14178562e-02,  1.17069989e-01,  1.49915814e-02,
       -1.57131720e-02, -4.93872985e-02,  2.56396998e-02,  7.85534531e-02,
        3.94085608e-02, -1.84973404e-02, -4.33945321e-02,  4.62529510e-02,
        4.40992303e-02, -6.88395798e-02,  5.51727526e-02, -4.77130376e-02,
       -1.38131222e-02, -6.33973554e-02, -4.98714931e-02,  4.12313081e-02,
       -4.18453403e-02, -4.09811549e-03, -6.12722449e-02,  4.84996960e-02,
       -3.21517102e-02,  2.92591024e-02, -5.57255857e-02,  1.41166588e-02,
       -7.48583600e-02,  3.48361954e-02, -1.57355085e-01, -5.03571965e-02,
        4.06579338e-02, -

# Zadanie

Twoim zadaniem jest dostarczenie możliwości wyszukiwania tekstów w wektorowej bazie danych.

1. Zapoznaj się z plikiem `movies.json`. Zawiera on dane testowe zawierające listę filmów wraz z krótkim opisem fabuły z repozytorium [erik-sytnyk/movies-list](https://github.com/erik-sytnyk/movies-list/blob/master/db.json).
2. Połącz się z Twoją instancją Qdrant.
3. Stwórz kolekcję, która przechowa wektory tekstów (w tym przypadku dobrą odległością będzie `COSINE`, bo podobne teksty układają się w *podobną stronę* w przestrzeni.
4. Wczytaj dane testowe, zwektoryzuj je i zapisz je w bazie danych.
5. Napisz funkcję, która na podstawie zadanej frazy zwróci pasujące filmy.

In [8]:
import json
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

In [41]:
with open("movies.json", "r", encoding="utf-8") as file:
    data = json.load(file)

movies = data["movies"]


formatted_movies = [
    {
        "id": movie.get("id", "Unknown"),
        "title": movie.get("title", "Unknown"),    
        "year": movie.get("year", "Unknown"),            
        "description": movie.get("plot", "No plot"),
        "director": movie.get("director", "No plot"),
        "actors": movie.get("actors", "No plot")      
    }
    for movie in movies
]

In [15]:
qdrant_client = QdrantClient(
    url="https://e15a0be7-90ca-491a-8bbb-ce2eae999b65.eu-west-2-0.aws.cloud.qdrant.io", 
    api_key="zQM1p21iOtdl60K2LiXMDwxtofTP3zXZxxmZsJGaGi1GWBF8VcFLlQ",
)

In [59]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

combined_data = [
    f"{movie['title']}: {movie['director']}: {movie['description']}" 
    for movie in formatted_movies
]

title = [movie["title"] for movie in formatted_movies]
vectors = encoder.encode(combined_data)

qdrant_client.recreate_collection(
    collection_name="movies",
    vectors_config=VectorParams(size=len(vectors[0]), distance=Distance.COSINE)
)

/tmp/ipykernel_7588/2253443379.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [60]:
movies_db = [
    {
        "id": movie["id"],                
        "vector": vector,                 
        "payload": {                      
            "title": movie["title"],
            "year": movie["year"],
            "description": movie["description"],
            "director": movie["director"],
            "actors": movie["actors"]
        }
    }
    for movie, vector in zip(formatted_movies, vectors)
]

In [61]:
qdrant_client.upsert(
    collection_name="movies",
    points=movies_db
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [57]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue

def search_movies(query: str, top_k: int = 5):


    query_vector = encoder.encode([query])[0]

    results = qdrant_client.search(
        collection_name="movies",
        query_vector=query_vector,
        limit=top_k 
    )

    movies = [
        {
            "title": result.payload["title"],
            "year": result.payload["year"],
            "description": result.payload["description"],
            "director": result.payload["director"],
            "actors": result.payload["actors"]
        }
        for result in results
    ]
    return movies

In [58]:
search_movies("Tim Burton", 10)

/tmp/ipykernel_7588/3881572148.py:8: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


[{'title': 'Alice in Wonderland',
  'year': '2010',
  'description': "Nineteen-year-old Alice returns to the magical world from her childhood adventure, where she reunites with her old friends and learns of her true destiny: to end the Red Queen's reign of terror.",
  'director': 'Tim Burton',
  'actors': 'Johnny Depp, Mia Wasikowska, Helena Bonham Carter, Anne Hathaway'},
 {'title': 'Beetlejuice',
  'year': '1988',
  'description': 'A couple of recently deceased ghosts contract the services of a "bio-exorcist" in order to remove the obnoxious new owners of their house.',
  'director': 'Tim Burton',
  'actors': 'Alec Baldwin, Geena Davis, Annie McEnroe, Maurice Page'},
 {'title': 'Corpse Bride',
  'year': '2005',
  'description': 'When a shy groom practices his wedding vows in the inadvertent presence of a deceased young woman, she rises from the grave assuming he has married her.',
  'director': 'Tim Burton, Mike Johnson',
  'actors': 'Johnny Depp, Helena Bonham Carter, Emily Watson, 